In [2]:
from curl_cffi import requests as cureq
import pandas as pd
import time
import json
from rich import print

In [50]:
response = cureq.get("https://www.sofascore.com/api/v1/unique-tournament/1644/season/7752/events/round/1", impersonate="chrome")

In [51]:
alldata =[]
games = json.loads(response.text)
for game in games["events"]:
    matchdata = {
                "tournament" : game["tournament"]["uniqueTournament"]["name"],
                "season" : game["season"]["year"],
                "round" : game["roundInfo"]["round"],
                "matchID" : game["id"],
                "matchCustomID" : game["customId"],
                "matchStatus" : game["status"]["description"], # Ended, Postponed, Not started
                "homeTeamnName" :game["homeTeam"]["name"],
                "homeTeamnNameCode":game["homeTeam"]["nameCode"],
                'homeTeamID' : game["homeTeam"]["id"],
                "awayTeamName": game["awayTeam"]["name"],
                'awayTeamNameCode': game["awayTeam"]["nameCode"],
                'awayTeamID' : game["awayTeam"]["id"],
    }
    
    if matchdata["matchStatus"] == "Ended":
        matchScores = {
            #'homeScoreHT' : game["homeScore"]["period1"],
            #'awayScoreHT' : game["awayScore"]["period1"],
            'homeScoreFT' : game["homeScore"]["normaltime"],
            'awayScoreFT' : game["awayScore"]["normaltime"],
        }
    else:
        matchScores = {
            #'homeScoreHT' : "-",
            #'awayScoreHT' : "-",
            'homeScoreFT' : "-",
            'awayScoreFT' : "-",
        }
    data = matchdata | matchScores
    alldata.append(data)


In [52]:
df = pd.DataFrame(alldata)
df

,tournament,season,round,matchID,matchCustomID,matchStatus,homeTeamnName,homeTeamnNameCode,homeTeamID,awayTeamName,awayTeamNameCode,awayTeamID,homeScoreFT,awayScoreFT
0,Kenyan Premier League,2014,1,5022474,JufbsNufb,Ended,Muhoroni Youth FC,MUY,138484,Sofapaka,SOF,138488,1,5
1,Kenyan Premier League,2014,1,5022472,PufbsUufb,Ended,Sony Sugar,SSU,138490,Top Fry Nakuru,TFN,138494,3,0
2,Kenyan Premier League,2014,1,5022476,BufbsYufb,Ended,Ulinzi Stars,ULS,138498,Chemelil Sugar,CSU,138476,1,1
3,Kenyan Premier League,2014,1,5022470,Hufbsavfb,Ended,Western Stima FC,WST,138500,Mathare United,MAU,138482,1,0
4,Kenyan Premier League,2014,1,5022478,zufbsFufb,Ended,Bandari FC,BAN,138474,Kenya Rev. Authority,KAU,138480,2,0
5,Kenyan Premier League,2014,1,5022480,RufbsWufb,Ended,Thika United,TUN,138492,Tusker,TUS,138496,1,1
6,Kenyan Premier League,2014,1,5022484,IDFsDufb,Ended,Kenya Commercial Bank,KCB,138478,AFC Leopards SC,LDS,76433,1,1
7,Kenyan Premier League,2014,1,5022482,gLdbsLufb,Ended,Gor Mahia FC,GOR,134306,Nairobi City Stars FC,NCS,138486,2,0
